In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
from pyspark.sql import functions as f
import pandas as pd

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Examen')\
        .getOrCreate()

22/08/24 14:48:16 WARN Utils: Your hostname, MacBook-Air-de-Willians.local resolves to a loopback address: 127.0.0.1; using 192.168.18.9 instead (on interface en0)
22/08/24 14:48:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/24 14:48:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/24 14:48:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/24 14:48:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Visualizando la data

In [4]:
dataCovid = spark.read.csv(
    'data/archive/covid_19_clean_complete.csv',
    sep = ',',
    header = True,
    )

dataCovid.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- WHO Region: string (nullable = true)



In [6]:
dataCovid.show(7)

+--------------+-------------------+--------+---------+----------+---------+------+---------+------+--------------------+
|Province/State|     Country/Region|     Lat|     Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|
+--------------+-------------------+--------+---------+----------+---------+------+---------+------+--------------------+
|          null|        Afghanistan|33.93911|67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|
|          null|            Albania| 41.1533|  20.1683|2020-01-22|        0|     0|        0|     0|              Europe|
|          null|            Algeria| 28.0339|   1.6596|2020-01-22|        0|     0|        0|     0|              Africa|
|          null|            Andorra| 42.5063|   1.5218|2020-01-22|        0|     0|        0|     0|              Europe|
|          null|             Angola|-11.2027|  17.8739|2020-01-22|        0|     0|        0|     0|              Africa|
|          null|Antigua 

#### Actualizando estructura de datos

In [8]:
from pyspark.sql.types import *

data_schema = [
               StructField('Province/State', StringType(), True),
               StructField('Country/Region', StringType(), True),
               StructField('Lat', DoubleType(), True),
               StructField('Long', DoubleType(), True),
               StructField('Date', DateType(), True),
               StructField('Confirmed', IntegerType(), True),
               StructField('Deaths', IntegerType(), True),
               StructField('Recovered', IntegerType(), True),
               StructField('Active', IntegerType(), True),
               StructField('WHO Region', StringType(), True),
            ]

final_struc = StructType(fields=data_schema)

In [11]:
dataCovid = spark.read.csv(
    'data/archive/covid_19_clean_complete.csv',
    sep = ',',
    header = True,
    schema = final_struc
    )

In [12]:
dataCovid.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)



In [13]:
dataCovid.show(5)

+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+
|Province/State|Country/Region|     Lat|     Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|
+--------------+--------------+--------+---------+----------+---------+------+---------+------+--------------------+
|          null|   Afghanistan|33.93911|67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|
|          null|       Albania| 41.1533|  20.1683|2020-01-22|        0|     0|        0|     0|              Europe|
|          null|       Algeria| 28.0339|   1.6596|2020-01-22|        0|     0|        0|     0|              Africa|
|          null|       Andorra| 42.5063|   1.5218|2020-01-22|        0|     0|        0|     0|              Europe|
|          null|        Angola|-11.2027|  17.8739|2020-01-22|        0|     0|        0|     0|              Africa|
+--------------+--------------+--------+---------+----------+---